<a href="https://colab.research.google.com/github/K4ly4s/K4ly4s/blob/main/%D0%9A%D0%BE%D0%BF%D0%B8%D1%8F_%D0%B1%D0%BB%D0%BE%D0%BA%D0%BD%D0%BE%D1%82%D0%B0_%22Image_to_Vector_Conversion_ipynb%22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Конвертация изображений в векторный формат
В этом ноутбуке мы реализуем код для конвертации растровых изображений в формате PNG в векторный формат SVG. Векторные изображения будут высокого качества с возможностью выделения объектов.

# Новый раздел

In [ ]:
!pip install -q opencv-python opencv-python-headless cairosvg svgwrite

## Импорт необходимых библиотек
Для начала импортируем все необходимые библиотеки. Мы будем использовать OpenCV для обработки изображений и CairoSVG для работы с SVG.

In [ ]:
import cv2
import cairosvg
import os
from tkinter import filedialog
from tkinter import Tk

## Функция для выбора папки с изображениями
Создадим функцию, которая открывает диалоговое окно для выбора папки, где хранятся изображения в формате PNG. Эта функция будет возвращать путь к выбранной папке.

In [ ]:
def select_folder():
    root = Tk()
    root.withdraw()  # Hide the main window
    folder_selected = filedialog.askdirectory()
    return folder_selected

## Функция для конвертации изображений в векторный формат
Теперь создадим функцию, которая принимает путь к папке с PNG-изображениями и конвертирует их в SVG-формат. В этой функции мы будем использовать OpenCV для обработки изображений и CairoSVG для создания SVG-файлов.

In [ ]:
def convert_images_to_svg(folder_path):
    for filename in os.listdir(folder_path):
        if filename.endswith('.png'):
            img_path = os.path.join(folder_path, filename)
            img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)  # Read the image in grayscale

            # Thresholding to get binary image
            _, thresh = cv2.threshold(img, 128, 255, cv2.THRESH_BINARY)

            # Initialize SVG file
            svg_filename = filename.replace('.png', '.svg')
            svg_path = os.path.join(folder_path, svg_filename)
            dwg = svgwrite.Drawing(svg_path, profile='tiny')

            # Iterate through the image and add paths to SVG
            for i in range(thresh.shape[0]):
                for j in range(thresh.shape[1]):
                    if thresh[i, j] == 0:  # Black pixel
                        dwg.add(dwg.rect((j, i), (1, 1), fill='black'))

            # Save SVG file
            dwg.save()
            print(f'Converted {filename} to {svg_filename}')

## Требования к векторизации
1. Контур и форма объектов должны быть максимально точными.
2. Необходимо сохранить цвет и градиенты.
3. Текстуры и узоры должны быть воспроизведены.
4. Необходима возможность редактировать отдельные объекты в векторном файле.
5. Векторный файл должен быть максимально детализирован.

## Реализация алгоритма векторизации
Исходя из вышеуказанных требований, нам нужно реализовать сложный алгоритм векторизации. Этот алгоритм будет включать в себя следующие шаги:
1. Чтение и предварительная обработка изображения.
2. Выделение контуров и объектов.
3. Воспроизведение цветов, градиентов и текстур.
4. Сохранение векторного изображения в формате SVG с возможностью редактирования отдельных объектов.
5. Оптимизация и детализация векторного изображения.

## Шаг 1: Чтение и предварительная обработка изображения
Первым шагом будет чтение изображения и его предварительная обработка. Это может включать в себя изменение размера изображения, преобразование в оттенки серого (если необходимо) и другие операции.

In [ ]:
def preprocess_image(image_path):
    # Чтение изображения
    img = cv2.imread(image_path)
    # Преобразование в RGB (OpenCV загружает изображения в BGR)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    # Ресайзинг
    img = cv2.resize(img, (300, 300))

    # Нормализация
    img = img / 255.0

    # Гауссова размытие
    img = cv2.GaussianBlur(img, (5, 5), 0)

    # Преобразование в оттенки серого
    img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)

    return img


## Шаг 2: Выделение контуров и объектов
На этом шаге мы будем использовать OpenCV для выделения контуров и объектов на изображении. Это необходимо для дальнейшей векторизации.

In [ ]:
def extract_contours(image):
    # Преобразование изображения в оттенки серого
    gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    # Применение пороговой фильтрации
    _, thresh = cv2.threshold(gray, 127, 255, cv2.THRESH_BINARY)
    # Поиск контуров
    contours, _ = cv2.findContours(image, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
    return contours

## Шаг 3: Воспроизведение цветов, градиентов и текстур
На этом шаге мы будем работать над сохранением цветов, градиентов и текстур при векторизации. Это может быть сложным процессом, в зависимости от сложности изображения.

In [ ]:
import cv2
import numpy as np

def reproduce_colors_and_textures(image, contours):
    # Копия изображения для рисования
    output = image.copy()

    for contour in contours:
        # Вычисление среднего цвета внутри контура
        mask = np.zeros(image.shape[:2], dtype=np.uint8)
        cv2.drawContours(mask, [contour], -1, 255, -1)
        mean_color = cv2.mean(image, mask=mask)

        # Заливка контура средним цветом
        cv2.drawContours(output, [contour], -1, mean_color, -1)

        # TODO: Добавить логику для градиентов и текстур если необходимо

    return output


## Шаг 4: Сохранение векторного изображения
После того как все объекты и их атрибуты (цвет, текстура и т.д.) будут обработаны, следующим шагом будет сохранение этих данных в формате SVG.

In [ ]:
def save_as_svg(contours, colors_and_textures, filename="output.svg"):
    # Инициализация SVG холста
    dwg = svgwrite.Drawing(filename, profile='tiny')

    for i, contour in enumerate(contours):
        # Извлечение цвета и текстуры (если есть)
        color = colors_and_textures[i]['color']
        texture = colors_and_textures[i].get('texture', None)

        # Преобразование контура в формат, подходящий для SVG
        contour = contour.astype(float).reshape(-1, 2)
        points = [tuple(point) for point in contour]

        # Создание полигона с контуром и цветом
        polygon = dwg.polygon(points=points, fill=color)
        dwg.add(polygon)

        # TODO: Добавить логику для текстур, если необходимо

    # Сохранение SVG файла
    dwg.save()

# Пример использования
contours = [np.array([[10, 10], [50, 10], [50, 50], [10, 50]])]
colors_and_textures = [{'color': 'red'}]
save_as_svg(contours, colors_and_textures)


## Шаг 5: Оптимизация и детализация
Последним шагом будет оптимизация и детализация векторного изображения. Это может включать в себя сглаживание контуров, оптимизацию количества точек в векторах и другие улучшения.

In [ ]:
from lxml import etree

def optimize_and_detail(svg_data):
    # Парсинг SVG данных
    root = etree.fromstring(svg_data)

    # Итерация по всем полигонам
    for polygon in root.xpath('//svg:polygon', namespaces={'svg': 'http://www.w3.org/2000/svg'}):
        points = polygon.attrib['points'].strip().split(' ')

        # TODO: Добавить логику для оптимизации и детализации

        # Пример: удаляем полигоны с менее чем 3 точками (не являются полигонами)
        if len(points) < 3:
            polygon.getparent().remove(polygon)

    # Сериализация обратно в строку
    optimized_svg_data = etree.tostring(root).decode('utf-8')
    return optimized_svg_data


In [ ]:
def reproduce_colors_and_textures(image, contours):
    # Инициализация словаря для хранения атрибутов каждого контура
    contour_attributes = {}

    for i, contour in enumerate(contours):
        # Вычисление центра масс контура для определения его цвета
        M = cv2.moments(contour)
        if M['m00'] != 0:
            cX = int(M['m10'] / M['m00'])
            cY = int(M['m01'] / M['m00'])
        else:
            cX, cY = 0, 0

        # Получение цвета в этой точке
        color = image[cY, cX]

        # TODO: Добавить логику для воспроизведения градиентов и текстур

        # Сохранение атрибутов контура
        contour_attributes[i] = {'color': color}

    return contour_attributes

In [ ]:
def reproduce_colors_and_textures(image, contours):
    # Инициализация словаря для хранения атрибутов каждого контура
    contour_attributes = {}

    for i, contour in enumerate(contours):
        # Вычисление центра масс контура для определения его цвета
        M = cv2.moments(contour)
        if M['m00'] != 0:
            cX = int(M['m10'] / M['m00'])
            cY = int(M['m01'] / M['m00'])
        else:
            cX, cY = 0, 0

        # Получение цвета в этой точке
        color = image[cY, cX]

        # Воспроизведение градиентов и текстур (простой пример)
        gradient = None  # TODO: Реализовать алгоритм для определения градиента
        texture = None  # TODO: Реализовать алгоритм для определения текстуры

        # Сохранение атрибутов контура
        contour_attributes[i] = {'color': color, 'gradient': gradient, 'texture': texture}

    return contour_attributes

In [ ]:
import numpy as np
import cv2

def detect_gradient(image, contour):
    # TODO: Реализовать алгоритм для определения градиента
    return None

def detect_texture(image, contour):
    # TODO: Реализовать алгоритм для определения текстуры
    return None

def reproduce_colors_and_textures(image, contours):
    # Инициализация словаря для хранения атрибутов каждого контура
    contour_attributes = {}

    for i, contour in enumerate(contours):
        # Вычисление центра масс контура для определения его цвета
        M = cv2.moments(contour)
        if M['m00'] != 0:
            cX = int(M['m10'] / M['m00'])
            cY = int(M['m01'] / M['m00'])
        else:
            cX, cY = 0, 0

        # Получение цвета в этой точке
        color = image[cY, cX]

        # Воспроизведение градиентов и текстур
        gradient = detect_gradient(image, contour)
        texture = detect_texture(image, contour)

        # Сохранение атрибутов контура
        contour_attributes[i] = {'color': color, 'gradient': gradient, 'texture': texture}

    return contour_attributes

In [ ]:
from skimage.feature import greycomatrix, greycoprops
from skimage import img_as_ubyte

def detect_gradient(image, contour):
    # Используем оператор Собеля для определения градиента
    sobelx = cv2.Sobel(image, cv2.CV_64F, 1, 0, ksize=3)
    sobely = cv2.Sobel(image, cv2.CV_64F, 0, 1, ksize=3)
    gradient = np.sqrt(sobelx**2 + sobely**2)
    return gradient

def detect_texture(image, contour):
    # Используем GLCM для определения текстуры
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    gray_image = img_as_ubyte(gray_image)
    glcm = greycomatrix(gray_image, [1], [0],  symmetric=True, normed=True)
    contrast = greycoprops(glcm, 'contrast')[0, 0]
    return contrast

def reproduce_colors_and_textures(image, contours):
    # Инициализация словаря для хранения атрибутов каждого контура
    contour_attributes = {}

    for i, contour in enumerate(contours):
        # Вычисление центра масс контура для определения его цвета
        M = cv2.moments(contour)
        if M['m00'] != 0:
            cX = int(M['m10'] / M['m00'])
            cY = int(M['m01'] / M['m00'])
        else:
            cX, cY = 0, 0

        # Получение цвета в этой точке
        color = image[cY, cX]

        # Воспроизведение градиентов и текстур
        gradient = detect_gradient(image, contour)
        texture = detect_texture(image, contour)

        # Сохранение атрибутов контура
        contour_attributes[i] = {'color': color, 'gradient': gradient, 'texture': texture}

    return contour_attributes